In [12]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [13]:
# prepare data
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

n_samples, n_features = X.shape
print(n_samples, n_features)

569 30


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1234)

# scale the feature to make logistic regression works better
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# convert to torch.tensor
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [15]:
# define the model
# f = wx + b, sigmoid at the end
class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
    
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return(y_pred)

model = LogisticRegression(n_features)

In [16]:
# define loss and optimizer
learning_rate = 0.01
criterion = nn.BCELoss() # Binary Cross-Entropy Loss
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [17]:
# construct training loop
n_epochs = 100

for epoch in range(n_epochs):
    # forward pass and loss calculation
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)
    
    # backward pass
    loss.backward()
    
    # update
    optimizer.step()
    
    # zero gradients
    optimizer.zero_grad()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch: {epoch+1}, loss = {loss:.4f}")
    
with torch.no_grad(): # detach from the optimization process for evaluation
    y_pred = model(X_test)
    y_pred_cls = y_pred.round() # 1 if y_pred > 0.5 else 0
    acc = y_pred_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f"Accuracy = {acc:.4f}")

Epoch: 10, loss = 0.5293
Epoch: 20, loss = 0.4472
Epoch: 30, loss = 0.3925
Epoch: 40, loss = 0.3533
Epoch: 50, loss = 0.3236
Epoch: 60, loss = 0.3003
Epoch: 70, loss = 0.2813
Epoch: 80, loss = 0.2655
Epoch: 90, loss = 0.2521
Epoch: 100, loss = 0.2405
Accuracy = 0.9211
